In [8]:
# Imports
import os
import torch.cuda, torch.utils.data, torch.nn, torch.optim, torch
import torchvision.transforms, torchvision.datasets.folder, torchvision.utils, torchvision.models
import transformers
import numpy as np
import json
import matplotlib.pyplot as plt
import time

In [9]:
# Addresses

class Address:
    def __init__(self):
        '''
        Stores all the addresses used in project
        '''
        # Inputs
        self.data = "../input/mammography"
        self.processed_data = "data.pkl"

        # Coco
        self.coco = os.path.join(self.data, 'coco_1k')
        self.coco_annot = os.path.join(self.coco, 'annotations')
        self.coco_annot_train = os.path.join(self.coco_annot, 'instances_train2017.json')
        self.coco_annot_val = os.path.join(self.coco_annot, 'instances_val2017.json')
        self.coco_img_train = os.path.join(self.coco, 'train2017')
        self.coco_img_val = os.path.join(self.coco, 'val2017')

        # Test
        self.test = os.path.join(self.data, 'test')
        self.test_img = os.path.join(self.test, 'images')
        self.test_label = os.path.join(self.test, 'labels')
        self.predictions = os.path.join(self.test, 'predictions')

        # Yolo
        self.yolo = os.path.join(self.data, 'yolo_1k')
        self.yolo_train = os.path.join(self.yolo, 'train')
        self.yolo_train_img = os.path.join(self.yolo_train, 'images')
        self.yolo_train_label = os.path.join(self.yolo_train, 'labels')
        self.yolo_val = os.path.join(self.yolo, 'val')
        self.yolo_val_img = os.path.join(self.yolo_val, 'images')
        self.yolo_val_labels = os.path.join(self.yolo_val, 'labels')

        # Models
        self.result = "results/"
        self.model_frcnn = os.path.join(self.result, 'frcnn')
        self.model_def_detr = os.path.join(self.result, 'deformable_dtr')

        # Temp
        self.temp = "temp/"

    def create_dir(self, dir_list = None):
        '''
        Function to create directories in dir_list. If dir_list is None then create all directories of address.
        '''
        if dir_list == None:
            dir_list = [self.temp, self.result, self.model_frcnn, self.model_def_detr]
        for address in dir_list:
            if not os.path.exists(address):
                os.mkdir(address)

    def _delete_folder_content(self, folder_addr):
        '''
        Deletes all the content of folder_addr
        '''
        if os.path.exists(folder_addr):
            for file in os.listdir(folder_addr):
                address = os.path.join(folder_addr, file)
                if os.path.isdir(address):
                    self._delete_folder_content(address)
                    os.removedirs(address)
                else:
                    os.remove(address)

    def clean(self, file_list = None):
        '''
        Deletes all the content in file_list
        '''
        if file_list == None:
            file_list = [self.temp]
        for address in file_list:
            self._delete_folder_content(address)

addr = Address()
addr.clean()
# addr.clean([addr.result])
addr.create_dir()

In [10]:
class HyperParameters:
    def __init__(self):
        '''
        Stores all Hyperparameters used for training of model
        '''
        # Training
        self.batch_size = 2
        self.num_epoch = 10
        self.grad_clip = 1.0

        # Data
        self.num_train = 40000
        self.num_val = 10000
        self.train_step = self.num_epoch*(self.num_train//self.batch_size)
        self.resolution = (512, 1024)
        
        # Learning Rate
        self.lr = 1e-4
        self.warmup_step = self.train_step//40
        self.decay_step = self.train_step//2
        self.decay_rate = 0.5

    def lr_schedule(self, step):
        '''
        Getting learning rate as function of train steps completed (Exponential decay with linear warmup)
        '''
        if step <= self.warmup_step:
            return step/self.warmup_step
        else:
            return self.decay_rate**((step-self.warmup_step)/self.decay_step)

    def create_report(self, addr):
        with open(os.path.join(addr, 'param.txt'), 'w') as file:
            file.writelines([
                f'Training:',
                f'\n\tBatch Size:       {self.batch_size}',
                f'\n\tNum Epoch:        {self.num_epoch}',
                f'\n\tGrad Clip:        {self.grad_clip}',
                f'\n\nData:',  
                f'\n\tNum Train:        {self.num_train}',
                f'\n\tNum Val:          {self.num_val}',
                f'\n\tTrain Step:       {self.train_step}',
                f'\n\tResolution:       {self.resolution}',
                f'\n\nLearning Rate:',  
                f'\n\tlr:               {self.lr}',
                f'\n\tWarmup Step:      {self.warmup_step}',
                f'\n\tDecay Step:       {self.decay_step}',
                f'\n\tDecay Rate:       {self.decay_rate}',
            ])

param = HyperParameters()

In [11]:
# Random Seed and CUDA

random_seed = 68
device = "cpu"
torch.manual_seed(random_seed)
np.random.seed(random_seed)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed_all(random_seed)
#     device = "cuda"
print(f"Working with device {device}")

Working with device cpu


In [23]:
# Dataset

class Data:
    def __init__(self, address: Address, param: HyperParameters):
        '''
        Creates DataLoader and DataSet for both train and val split
        '''
        self.address = address
        self.param = param

        # Transform
        self.transform = torchvision.transforms.Compose([torchvision.transforms.Resize(param.resolution),
                                                         torchvision.transforms.ToTensor()])

        # Dataset
        self.dataset_train = torchvision.datasets.CocoDetection(self.address.coco_img_train,
                                                                self.address.coco_annot_train,
                                                                transform=self.transform,
                                                                target_transform=self.trg_transform)
        self.dataset_val = torchvision.datasets.CocoDetection(self.address.coco_img_val,
                                                              self.address.coco_annot_val,
                                                              transform=self.transform,
                                                              target_transform=self.trg_transform)

        # DataLoader
        self.loader_train = torch.utils.data.DataLoader(self.dataset_train, batch_size=param.batch_size, shuffle=True)
        self.loader_val = torch.utils.data.DataLoader(self.dataset_val, batch_size=param.batch_size, shuffle=False)
    
    def trg_transform(self, elem):
        if 'bbox' in elem:
            return_dict = {'bbox': torch.tensor(elem['bbox'])}
            return_dict['bbox'][2] += return_dict['bbox'][0]
            return_dict['bbox'][3] += return_dict['bbox'][1]
            return return_dict
        return {'bbox': torch.tensor([]).reshape(0)}
  
    def collate(self, batch):
        img_list = []
        trg_list = {'bbox': []}
        for elem in batch:
            img_list.append(elem[0])
            trg_list['bbox'].append(elem[1]['bbox'])
        trg_list['bbox'] = torch.stack(trg_list['bbox'])
        return torch.stack(img_list), trg_list

data = Data(addr, param)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [24]:
# Model

# Define Deformable DETR model configuration
config = transformers.DeformableDetrConfig.from_pretrained("SenseTime/deformable-detr", num_labels=2)

# Initialize Deformable DETR model
model = transformers.DeformableDetrModel(config)

# Training arguments
model_addr = os.path.join(addr.model_def_detr, 'model')
log_addr = os.path.join(addr.model_def_detr, 'logs')
addr.create_dir([model_addr, log_addr])
training_args = transformers.TrainingArguments(output_dir = model_addr,
                                               num_train_epochs = param.num_epoch,
                                               per_device_train_batch_size = param.batch_size,
                                               learning_rate = param.lr,
                                               warmup_steps = param.warmup_step,
                                               evaluation_strategy ='epoch',
                                               save_strategy = 'epoch',
                                               logging_dir = log_addr,
                                               logging_strategy = 'steps',
                                               logging_steps = 1,
                                               use_cpu = False,
                                               seed = random_seed,
                                               load_best_model_at_end =True)

# Define Trainer
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=data.dataset_train,
    eval_dataset=data.dataset_val,
    data_collator=data.collate
)

# Start training
trainer.train()

c:\Users\ris04\AppData\Local\Programs\Python\Python38\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  0%|          | 0/11200 [01:01<?, ?it/s]


TypeError: DeformableDetrModel object argument after ** must be a mapping, not list